In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cri98\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#df = pd.read_csv("selected_patents.csv").drop_duplicates()
df = pd.read_csv("../data/selected_patent_final.csv").drop_duplicates()

df.claims = df.claims.fillna("")

df.head()

,filename,filing_date,publication_date,priority_date,ipc_classes,assignee,inventors,docdb_family_id,title,abstract,claims
0,US2019350514,2019-06-14,2019-11-21,2018-05-17,"A45D44/00,A61B5/00,G06N3/02,G06Q30/06",P&G (PROCTER & GAMBLE COMPANY),"KAINKARYAM, RAGHUNANDAN MELKOTEPURWAR, ANKURSH...",68534608,systems and methods for hair analysis,disclosed are hair analysis systems and method...,1. a hair analysis system comprising: (a) an i...
1,WO2019222327,2019-05-15,2019-11-21,2018-05-17,G06Q30/06,P&G (PROCTER & GAMBLE COMPANY),"SHERMAN, FAIZPURWAR, ANKURKAINKARYAM, RAGHUNANDAN",66770570,systems and methods for hair analysis,disclosed are hair analysis systems and method...,
2,US2019325200,2019-06-28,2019-10-24,2017-08-09,"G06K9/00,G06K9/34,G06K9/62",BEIJING SENSETIME TECHNOLOGY DEVELOPMENT COMPANY,"QIAN CHENWANG FEIWANG, QUAN",62654894,"face image processing methods and apparatuses,...",a face image processing method includes: segme...,"1. a face image processing method, comprising:..."
3,EP3769852,2019-07-22,2021-01-27,2019-07-22,"A47K5/12,B05B11/00,B05B12/00,B05B15/30,H02N2/18","Open Innovation 2 Go, SL","Baturone Moreno de Carlos, PabloAbad de Aranzá...",67480134,pump- actuated dispenser with integrated switc...,the invention refers to a pump-actuated dispen...,1. a pump-actuated dispenser (1) comprising: a...
4,US10897979,2019-09-12,2021-01-26,2019-09-12,"A45D44/00,A61Q5/10,G05B17/02,G06K9/46,G06T11/0...",SURETINT TECHNOLOGIES,"Byrne, ChristopherNash, Cody Z.Christie, Eliza...",74191038,system and method for hair dye color conversion,a system and method is provided for hair dye c...,1. a method for hair dye color conversion in w...


In [72]:
columns = df.columns.to_list()
tokeep = [True for _ in range(len(df))]

df = df.sort_values(by=["docdb_family_id", "claims"])

dists = [0.0]

pbar = tqdm(zip(df.values[:-1], df.values[1:]), total=len(df)-1)

for i, (row1, row2) in enumerate(pbar):
    pbar.set_description(f'{row1[columns.index("docdb_family_id")]} vs {row2[columns.index("docdb_family_id")]}')
    
    print(f'{row1[columns.index("docdb_family_id")]} vs {row2[columns.index("docdb_family_id")]}')
    
    if row1[columns.index("docdb_family_id")] != row2[columns.index("docdb_family_id")]:
        #pass
        dists.append(None)
        continue
    
    #dist = nltk.edit_distance(row1[columns.index("claims")], row2[columns.index("claims")])
    try:
        dist = nltk.jaccard_distance(set(nltk.word_tokenize(row1[columns.index("claims")])), 
                                     set(nltk.word_tokenize(row2[columns.index("claims")])))
    except:
        dists.append(None)
        tokeep[i] &= False
        continue
        
    dists.append(dist)
    print(f'({row1[columns.index("docdb_family_id")]})jaccard distance: {dist}')
    if dist < .8:
        tokeep[i] &= False

  0%|          | 0/309 [00:00<?, ?it/s]

21820138 vs 21820158
21820158 vs 22926956
22926956 vs 26698498
26698498 vs 26698498
(26698498)jaccard distance: 0.47257383966244726
26698498 vs 26698498
(26698498)jaccard distance: 0.4567627494456763
26698498 vs 26990036
26990036 vs 27362297
27362297 vs 35061324
35061324 vs 35923560
35923560 vs 36092105
36092105 vs 37964796
37964796 vs 38369411
38369411 vs 38476897
38476897 vs 38801014
38801014 vs 38801014
(38801014)jaccard distance: 0.09947643979057591
38801014 vs 38801014
(38801014)jaccard distance: 0.44787644787644787
38801014 vs 39609342
39609342 vs 39745040
39745040 vs 39745040
(39745040)jaccard distance: 0.375
39745040 vs 39745040
(39745040)jaccard distance: 0.3069306930693069
39745040 vs 39745040
(39745040)jaccard distance: 0.2192513368983957
39745040 vs 40588129
40588129 vs 40588129
(40588129)jaccard distance: 0.5263157894736842
40588129 vs 40588129
(40588129)jaccard distance: 0.46994535519125685
40588129 vs 40588129
(40588129)jaccard distance: 0.20118343195266272
40588129 vs 4

In [73]:
df["to_keep"] = tokeep
df["dist"] = dists

In [74]:
import plotly.express as px

px.line(df.sort_values(by="dist"), x=range(len(df)), y="dist").show()

In [75]:
import plotly.express as px
df_sorted = df.sort_values(by="dist")
px.line(df_sorted.reset_index(drop=True).reset_index(), x="dist", y="index").show()

In [76]:
df_sorted

,filename,filing_date,publication_date,priority_date,ipc_classes,assignee,inventors,docdb_family_id,title,abstract,claims,to_keep,dist
321,US2003120550,2001-12-21,2003-06-26,2001-12-21,"G06F17/30,G06Q30/06",L'OREAL,PEYRELEVADE JEROMEPOUTEAU ARNOLDBURGARD DOMINQUE,21820138,shop-in-shop website construction,"systems, methods, and products may employ a mo...",1. a method for facilitating modular construct...,True,0.0
242,US2017251130,2017-02-10,2017-08-31,2016-02-29,"A45D40/30,A45D44/00,A61F13/00,A61K8/02,A61Q1/0...",PANASONIC INTELLECTUAL PROPERTY,"SHINODA, MASAYO",58094194,image processing apparatus and method to gener...,an image processing apparatus generates image ...,1. an image processing apparatus that generate...,False,0.0
300,WO2010093503,2010-01-20,2010-08-19,2007-01-05,"A61B6/00,G01N21/21","MYSKINMEHENDALE, RAHULMARINKOVICH, SAVAKORUGA,...","MEHENDALE, RAHULMARINKOVICH, SAVAKORUGA, DJURO...",42562243,skin analysis methods,the present invention generally relates to a s...,"what is claimed is:1. a skin care device, the ...",True,0.0
230,US10249050,2017-09-15,2019-04-02,2016-09-27,"G06T7/00,G06T7/11,G06T7/12,G06T7/149,G06T7/174...",CANON,"ENDO TAKAAKINAKANO, YUTASATOH, KIYOHIDE",61687337,image processing apparatus and image processin...,an image processing apparatus includes an imag...,1. an image processing apparatus that aligns a...,True,0.0
216,US10491813,2019-05-15,2019-11-26,2016-12-07,"A61B1/00,G06K9/32,G06T7/11,H04N5/232",OLYMPUS CORPORATION,"SHIRATANI, FUMIYUKI",62492311,image processing apparatus and image processin...,an image processing apparatus includes an imag...,1. an image processing apparatus comprising a ...,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,US2020399558,2020-06-17,2020-12-24,2019-06-21,"A23L27/00,C11B9/00",UNIVERSITY OF CALIFORNIA,"RAY, ANANDASANKARKOWALEWSKI, Joel",74039135,"methods for identifying, compounds identified ...",provided herein are screening methods for iden...,"1. a flavor/fragrance composition, comprising ...",True,NaN
7,US2020405037,2019-06-26,2020-12-31,2019-06-26,A45D34/04,L'OREAL,"BICKFORD, WILLIAMEl-Sayed, RogenaFodor, Nicholas",74043320,cosmetic package including a liquid reservoir,cosmetic formula containers generally include ...,the embodiments of the invention in which an e...,True,NaN
180,US2020410755,2020-06-15,2020-12-31,2019-06-25,"G06K9/00,G06T19/00,G10L15/08",SONY INTERACTIVE ENTERTAINMENT,"NAKATA, MASASHI",74044172,image processing apparatus and image processin...,an image processing apparatus includes an imag...,1. an image processing apparatus comprising: a...,True,NaN
6,US2021000243,2019-07-02,2021-01-07,2019-07-02,"A45D44/00,G06F9/54,G06Q10/08",Gemma Labs Inc.,"LOVING, ADAM DEANRush, Richard BenjaminHarr, A...",74066188,personal care product recommendation platform,disclosed is a personal care product platform ...,1. a system for recommending personal care pro...,True,NaN


In [80]:
len(df)

310

In [81]:
len(df[df.to_keep == True])

204

In [82]:
df[df.to_keep == True].sort_values(by="docdb_family_id")[['title', 'docdb_family_id', "claims", "dist"]]

,title,docdb_family_id,claims,dist
321,shop-in-shop website construction,21820138,1. a method for facilitating modular construct...,0.000000
319,cosmetic affinity indexing,21820158,1. a method of helping a consumer make a cosme...,NaN
322,method of evaluating cosmetic products on a co...,22926956,1. a method for demonstrating effectiveness of...,NaN
29,body image enhancement,26698498,"a method of constructing a body image, the met...",0.456763
318,complete compact,26990036,i claim:1. a method for determining aroma comp...,NaN
...,...,...,...,...
8,"methods for identifying, compounds identified ...",74039135,"1. a flavor/fragrance composition, comprising ...",NaN
7,cosmetic package including a liquid reservoir,74043320,the embodiments of the invention in which an e...,NaN
180,image processing apparatus and image processin...,74044172,1. an image processing apparatus comprising: a...,NaN
6,personal care product recommendation platform,74066188,1. a system for recommending personal care pro...,NaN


In [83]:
df_tmp = df[df.to_keep == True][["docdb_family_id", "filename", "title", "claims"]].copy()
#df_tmp.filename = df_tmp.filename.apply(lambda x: x[:-7])
df_tmp

,docdb_family_id,filename,title,claims
321,21820138,US2003120550,shop-in-shop website construction,1. a method for facilitating modular construct...
319,21820158,US2003120534,cosmetic affinity indexing,1. a method of helping a consumer make a cosme...
322,22926956,US2002054714,method of evaluating cosmetic products on a co...,1. a method for demonstrating effectiveness of...
29,26698498,EP1298587,body image enhancement,"a method of constructing a body image, the met..."
318,26990036,US2003127105,complete compact,i claim:1. a method for determining aroma comp...
...,...,...,...,...
8,74039135,US2020399558,"methods for identifying, compounds identified ...","1. a flavor/fragrance composition, comprising ..."
7,74043320,US2020405037,cosmetic package including a liquid reservoir,the embodiments of the invention in which an e...
180,74044172,US2020410755,image processing apparatus and image processin...,1. an image processing apparatus comprising: a...
6,74066188,US2021000243,personal care product recommendation platform,1. a system for recommending personal care pro...


In [84]:
df_tmp.to_excel("selected_patents_noDuplicati.xlsx", index=False)

In [69]:
v1="1. an image processing apparatus comprising: target finding acquisition means (1002) for acquiring a target finding that is selected from a plurality of imaging findings and that is included in a medical image; type identification means (1003) for acquiring a type of image feature corresponding to the acquired target finding, from types of image feature; feature quantity calculation means (1004) for calculating an image feature quantity of the acquired type for each of a plurality of cross-sectional images of the medical image; and cross-sectional image identification means (1005) for identifying one of the plurality of cross-sectional images based on the calculated image feature quantity, wherein the imaging findings are associated in advance with the types of image feature, and wherein the imaging findings and the target finding are expressed as words.2. the image processing apparatus according to claim 1, wherein the target finding acquisition means is configured to acquire, as the target finding, an imaging finding input by a user.3. the image processing apparatus according to claim 2, wherein the target finding acquisition means further comprises inference means for inputting a plurality of imaging findings to infer a diagnosis and calculating contribution of each of a plurality of the imaging findings to an inferred diagnosis and is configured to select one of the plurality of imaging findings that backs up diagnosis as the target finding based on the contribution to the inferred diagnosis calculated by the inference means.4. the image processing apparatus according to claim 1, further comprising a display control means (1006) for displaying, on a monitor (104), an interface (if3) for inputting at least one target finding from a pull-down menu.5. the image processing apparatus according to claim 1, further comprising:image acquisition means (1001) for acquiring a medical image for diagnosis, from a picture archiving and communicating system.6. the image processing apparatus according to claim 1, wherein the cross-sectional image identification means is operable to calculate a value for selecting a cross-sectional image, for the acquired target finding.7. the image processing apparatus according to claim 1, wherein the target finding acquisition means is operable to acquire a plurality of target findings, and the cross-sectional image identification means is operable to select cross sectional images each corresponding to different one of the acquired target findings.8. the image processing apparatus according to claim 7, wherein a cross-sectional image is selectable from the selected cross sectional images by a user.9. the image processing apparatus according to claim 1, wherein the cross-sectional image identification means (1005) is configured to identify the one cross-sectional image having the calculated image feature quantity corresponding to a representative characteristic value, with a significant difference from an image feature quantity corresponding to an evaluation value of the target finding, wherein the evaluation value is input by a user.10. the image processing apparatus according to claim 1, further comprising graph generation means (1203) for generating a graph that provides a comparison between the image feature quantities of the plurality of cross-sectional images calculated by the feature quantity calculation means (1004) and the target findings.11. the image processing apparatus according to claim 10, wherein the graph generation means (1203) is configured to generate the graph such that the image feature quantity and the target finding are plotted, for each of a plurality of tomographic images of a three-dimensional image, on coordinates of a first axis representing the image feature quantity and the target finding and a second axis, different from the first axis, representing positions of the plurality of tomographic images.12. an image processing method comprising the steps of: acquiring (s420) a target finding that is selected from a plurality of imaging findings and that is included in a medical image; acquiring (s430) a type of image feature corresponding to the acquired target finding, from types of image feature; calculating (s440) an image feature quantity of the acquired type for each of a plurality of cross-sectional images of the medical image; and identifying (s450) one of the plurality of cross-sectional images based on the calculated image feature quantity, wherein the imaging findings are associated in advance with the types of image feature, and wherein the imaging findings and the target finding are expressed as words.13. a computer program comprising instructions which, when the program is executed by a computer, cause the computer to carry out the method according to claim 12.14. a computer-readable storage medium comprising instructions which, when executed by a computer, cause the computer to carry out the method according to claim 12."

In [70]:
v2="an image processing apparatus comprising: target finding acquisition means for acquiring a target finding of a medical image; feature quantity calculation means for calculating an image feature quantity of a type associated in advance with the target finding acquired by the target finding acquisition means, for each of a plurality of cross-sectional images of the medical image; and cross-sectional image identification means for identifying one of the plurality of cross-sectional images based on the image feature quantity calculated by the feature quantity calculation means.the image processing apparatus according to claim 1, wherein the target finding acquisition means is configured to automatically select the target finding based on the image feature quantity.the image processing apparatus according to claim 1, wherein the target finding acquisition means is configured to acquire, as the target finding, an imaging finding input by a user.the image processing apparatus according to claim 1, wherein the target finding acquisition means further comprises inference means for inputting a plurality of target findings to infer a diagnosis and is configured to select one of the plurality of target findings based on contribution to the inference by the inference means.an image processing method comprising the steps of: acquiring a target finding of a medical image; calculating an image feature quantity of a type associated in advance with the acquired target finding, for each of a plurality of cross-sectional images of the medical image; and identifying one of the plurality of cross-sectional images based on the calculated image feature quantity.a diagnostic imaging support apparatus comprising: imaging finding acquisition means for acquiring, from a user, a type and an evaluation value of an imaging finding related to a predetermined three-dimensional region of a three-dimensional image to be diagnosed; characteristic value calculation means for calculating a characteristic value that can be compared with the evaluation value of the imaging finding based on an image feature, from each of a plurality of cross-sectional images included in the predetermined three-dimensional region; representative characteristic value calculation means for calculating a representative characteristic value that represents the predetermined three-dimensional region from a plurality of characteristic values calculated from the plurality of cross-sectional images; representative cross-sectional image acquisition means for acquiring a cross-sectional image, in which the  representative characteristic value is calculated, as a representative cross-sectional image from the plurality of cross-sectional images; comparison means for comparing the evaluation value of the imaging finding and the representative characteristic value; and display means for displaying the representative cross-sectional image only if a comparison result of the comparison unit indicates a significant difference.the diagnostic imaging support apparatus according to claim 6, further comprising: approximate characteristic value selection means for selecting an approximate characteristic value closest to the evaluation value of the imaging finding from the plurality of characteristic values calculated from the plurality of cross-sectional images; and approximate cross-sectional image acquisition means for acquiring a cross-sectional image, in which the approximate characteristic value is calculated, as an approximate cross-sectional image from the plurality of cross-sectional images, wherein the display means is configured to display both the approximate cross-sectional image and the representative cross-sectional image only if the comparison result of the comparison means indicates a significant difference.the diagnostic imaging support apparatus according to claim 6 or 7, wherein  the plurality of cross-sectional images comprises a plurality of cross-sectional images in each of a plurality of predetermined directions, and the representative cross-sectional image acquisition means and/or the approximate cross-sectional image acquisition means can acquire the representative cross-sectional image and/or the approximate cross-sectional image from cross-sectional images in an arbitrary direction.a diagnostic imaging support method comprising the steps of: acquiring, from a user, a type and an evaluation value of an imaging finding related to a predetermined three-dimensional region of a three-dimensional image to be diagnosed; calculating a characteristic value that can be compared with the evaluation value of the imaging finding based on an image feature, from each of a plurality of cross-sectional images included in the predetermined three-dimensional region; calculating a representative characteristic value that represents the predetermined three-dimensional region from a plurality of characteristic values calculated from the plurality of cross-sectional images; acquiring a cross-sectional image, in which the representative characteristic value is calculated, as a representative cross-sectional image from the plurality of cross-sectional images;  comparing the evaluation value of the imaging finding and the representative characteristic value; and displaying the representative cross-sectional image only if a comparison result in the comparing step indicates a significant difference.a diagnostic imaging support apparatus comprising: information acquisition means for acquiring information including a target finding item and a value of finding of the target finding item from a plurality of finding items in relation to a three-dimensional image to be diagnosed; feature quantity calculation means for calculating a feature quantity corresponding to the target finding item from each of a plurality of cross-sectional images that form the three-dimensional image; and graph generation means for generating a graph based on the feature quantity calculated by the feature quantity calculation means and the value of finding.the diagnostic imaging support apparatus according to claim 10, wherein the graph generation means is configured to display the value of finding in a band shape.a diagnostic imaging support method comprising the steps of: acquiring information including a target finding item and a value of finding of the target finding item from a  plurality of finding items in relation to a three-dimensional image to be diagnosed; calculating a feature quantity corresponding to the target finding item from each of a plurality of cross-sectional images that form the three-dimensional image; and generating a graph based on the calculated feature quantity and the value of finding.the diagnostic imaging support method according to claim 12, wherein the generating step is configured to display the value of finding in a band shape.a computer program comprising processor implementable steps for performing a method according to any one of claims 5, 9, 12 and 13.a computer software product storing a computer program according to claim 14."

In [71]:
nltk.jaccard_distance(set(nltk.word_tokenize(v1)), set(nltk.word_tokenize(v2)))

0.6108597285067874

In [68]:
nltk.jaccard_distance({"ciao"}, {"mamma"})

1.0

In [83]:
df[df.to_keep == True]["abstract"]

266    systems, methods, and products may employ a mo...
264    methods and systems are disclosed for helping ...
267    a method kit is provided for demonstrating the...
29     systems, methods, and apparatus consistent wit...
28     systems, methods, and apparatus consistent wit...
                             ...                        
7      provided herein are screening methods for iden...
6      cosmetic formula containers generally include ...
142    an image processing apparatus includes an imag...
5      disclosed is a personal care product platform ...
3      a system and method is provided for hair dye c...
Name: abstract, Length: 225, dtype: object